# Translation Group

Here the implementations and mechanisms of translation group module are demonstrated with various examples.


In [1]:
import numpy as np
from principia_materia import Fraction
from principia_materia.io_interface import parse_array
from principia_materia.translation_group import (
    get_structure,
    save_structure,
    Lattice,
    LatticeFTG,
    Cluster,
    Crystal,
    CrystalFTG,
    Kpoints,
    QpointsN,
    get_minimum_supercell,
)


## Cluster

Here an Cluter of object of a $CO_2$ molecule is used for demonstration.
Below is a simple diagram of the molecule.

![co2](illustrations/molecule/co2.pdf)

In [2]:
atoms = {
    "C": np.array([
        [0.0, 0.0, 0.0]
    ]),
    "O": np.array([
        [ 1.0, 0.0, 0.0],
        [-1.0, 0.0, 0.0]
    ])
}
cluster = Cluster(atoms=atoms, orbitals="p_x,p_y")

Here we set the orbitals to $p_x$ and $p_y$ to account for both along-the-axis and perpendicular-to-axis displacements.

Then we are able do many things with the cluster object.

For example, we can rotate the cluster by 90 degrees.

In [3]:
rotation_90 = np.array([
    [0, -1, 0],
    [1,  0, 0],
    [0,  0, 1],
], dtype=float)
example_cluster = cluster.copy()
example_cluster.rotate(rotation_90)
print(example_cluster.atoms)

OrderedDict([('C', array([[0., 0., 0.]])), ('O', array([[ 0.,  1.,  0.],
       [ 0., -1.,  0.]]))])


Initially, we put the origin of the axis for the positions of the cluter at the center of it.

In [4]:
cluster.center

array([0., 0., 0.])

We can move the cluster to set the origin at one of the $O$ atoms.

In [5]:
example_cluster = cluster.copy()
example_cluster.shift_atoms([1, 0, 0])
print("center:", example_cluster.center)
print("positions:")
print(example_cluster.atoms)

center: [1. 0. 0.]
positions:
OrderedDict([('C', array([[1., 0., 0.]])), ('O', array([[2., 0., 0.],
       [0., 0., 0.]]))])


## Lattice

Lattice is an infinite array of points generated by translation symmetry from the primitive traslation vectors (lattice vectors).

Here the lattice vectors of face center cubic is used as a demonstration.

In [6]:
lattice_vectors = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
lattice = Lattice(vec=lattice_vectors)

With the Lattice object, we can compute various properties of the lattice.

In [7]:
print("volume:", lattice.vol)
print("lengths of the lattice vectors:", lattice.abc)
print("angles between the lattice vectors:", lattice.abg)

volume: 0.25
lengths of the lattice vectors: [0.70710678 0.70710678 0.70710678]
angles between the lattice vectors: [60. 60. 60.]


With lattice comes a reciprocal lattice. We can also compute the properties of the reciprocal lattice.

In [8]:
print("volume of reciprocal lattice:", lattice.rvol)
print("lengths of the reciprocal lattice vectors:", lattice.rabc)
print("angles between the reciprocal lattice vectors:", lattice.rabg)

volume of reciprocal lattice: 992.200854
lengths of the reciprocal lattice vectors: [10.88279619 10.88279619 10.88279619]
angles between the reciprocal lattice vectors: [109.47122063 109.47122063 109.47122063]


Note that the reciprocal lattice vectors contain the $2\pi$ factor.

In [9]:
np.dot(lattice.vec, lattice.rvec)

array([[6.28318531, 0.        , 0.        ],
       [0.        , 6.28318531, 0.        ],
       [0.        , 0.        , 6.28318531]])

We can apply strain to an axial lattice to break it's point symmetry.

In [10]:
strain = [0.02, 0, 0] # axial strain along x
example_lattice = lattice.copy()
example_lattice.axial_strain(strain)
print(example_lattice.vec)

[[0.   0.5  0.5 ]
 [0.51 0.   0.5 ]
 [0.51 0.5  0.  ]]


## Crystal and CrystalFTG

A crystal is constructed by putting a cluster of atoms into the lattice.
Accordingly, we created the Crystal class by deriving from Lattice and Cluster at the same time.

Due to limitations of the complexity and computing power, we are unable to work with the infinite lattice.
Thus we often turn to a finite translation group as an approximation.

A finite translation group is defined by a supercell with which there are a finite number of translation vectors that translate the primitive cell of the crystal across the space.

First we start by constructing the primitive crystal for $Na Cl$.

In [11]:
lattice_vectors = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
atoms = {
    "Na": np.array([[0.0, 0.0, 0.0]]),
    "Cl": np.array([[0.5, 0.5, 0.5]]),
}
crystal = Crystal(vec=lattice_vectors, atoms=atoms)

In [12]:
crystal.to_dict()

OrderedDict([('vec',
              array([[0. , 0.5, 0.5],
                     [0.5, 0. , 0.5],
                     [0.5, 0.5, 0. ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', None)])

In [13]:
rotation_90 = np.array([
    [0, -1, 0],
    [1,  0, 0],
    [0,  0, 1],
], dtype=float)

In [14]:
example_crystal = crystal.copy()
print(example_crystal.positions_cartesian)
print(example_crystal.rotate_atoms(rotation_90))

[[0.  0.  0. ]
 [0.5 0.5 0.5]]
[[ 0.   0.   0. ]
 [ 1.5 -0.5 -0.5]]


Then we create the crystal with a FTG of $4\hat{1}$.

In [29]:
supa = np.identity(crystal.dim, dtype=int) * 2
supa

array([[2, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])

In [30]:
crystal_ftg = CrystalFTG.from_primitive(crystal, supa)
crystal_ftg.to_dict()

OrderedDict([('vec',
              array([[0. , 0.5, 0.5],
                     [0.5, 0. , 0.5],
                     [0.5, 0.5, 0. ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', None),
             ('supa',
              array([[2, 0, 0],
                     [0, 2, 0],
                     [0, 0, 2]]))])

Noted that throughout this software suite, we commonly use the CrystalFTG object with the identity $\hat{1}$ supercell as the primitive cell for convenience of rescaling the supercell at any time.

### Wigner Seitz cell of a supercell

In [34]:
ws_cell_translation, ws_cell_atoms, ws_cell_weights = \
    crystal_ftg.get_wigner_seitz_cell(center=[0, 0, 0])

In [36]:
ws_cell_weights

array([1.        , 0.5       , 0.5       , 0.5       , 0.25      ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.25      , 0.25      , 0.25      , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.25      ,
       0.25      , 0.25      , 0.25      ])

### Displacements basis vectors at a given q-point

These displacements basis vectors are the naive basis of nuclei displacements in reciprocal space, which is crucial to lattice dynamics analysis.

In [32]:
qpoint = parse_array("1/2 0 0", dtype=Fraction)
qpoint

[Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)]

In [33]:
crystal_ftg.orbitals = "p"
basis = crystal_ftg.get_basis_at_q(qpoint)
basis[(0, "p_x")].real

array([[ 0.35355339,  0.        ,  0.        ],
       [-0.35355339,  0.        ,  0.        ],
       [ 0.35355339,  0.        ,  0.        ],
       [-0.35355339,  0.        ,  0.        ],
       [ 0.35355339,  0.        ,  0.        ],
       [-0.35355339,  0.        ,  0.        ],
       [ 0.35355339,  0.        ,  0.        ],
       [-0.35355339,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

## Algorithm to find translation vectors of any given supercell

Meanwhile, although the supercell matrices are most commonly perceived as diagonal matrices.
Non-diagonal supercells are crucial in many parts of this software suite, particularly the lattice dynamic analysis of crystals. Thus a robust algorithm is needed to find the translation vectors of any given supercell matrix, as well as indexing the translation vectors within any given supercell.

The details of the algorithm to find translation vectors of any given supercell is explained in paper
[Phys. Rev. B 100, 014303 (2019)](https://doi.org/10.1103/PhysRevB.100.014303)
The same algorithm can be reversed to compute the indices of the lattice points at $\mathcal{O}(1)$ without seaching through the list of all translation vectors.

In [23]:
non_diagonal_supa = np.ones((3, 3), dtype=int) - 2 * np.identity(3, dtype=int)

In [24]:
lattice_ftg = LatticeFTG(np.identity(3), non_diagonal_supa)

In [25]:
lattice_ftg.lattice_points

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

In [26]:
lattice_ftg.get_index(lattice_ftg.lattice_points)

array([0, 1, 2, 3])

## Algorithm to find the minimum supcell for a given list of q-points

See derivtion also in [Phys. Rev. B 100, 014303 (2019)](https://doi.org/10.1103/PhysRevB.100.014303)

In [37]:
Qpoint = parse_array("""\
    1/4 3/4 0
""", dtype=Fraction)
Qpoint

[Fraction(1, 4), Fraction(3, 4), Fraction(0, 1)]

In [38]:
get_minimum_supercell(Qpoint)

array([[1, 1, 0],
       [0, 4, 0],
       [0, 0, 1]])

In [39]:
Qpoint = parse_array("""\
    1/4 3/4 0
    1/4 1/2 0
    3/4 1/4 1/2
""", dtype=Fraction)
Qpoint

[[Fraction(1, 4), Fraction(3, 4), Fraction(0, 1)],
 [Fraction(1, 4), Fraction(1, 2), Fraction(0, 1)],
 [Fraction(3, 4), Fraction(1, 4), Fraction(1, 2)]]

In [40]:
get_minimum_supercell(Qpoint)

array([[4, 0, 0],
       [0, 4, 0],
       [0, 0, 2]])

## Q-points at a given order and FTG

Phonon interactions are associated with Q-points at the order of the interaction.
Here we need an efficient algorithm to find symmetrically irreducible Q-points in a given FTG and order.

The first step is to find the irreducible k-points.
We continue the $NaCl$ example.

In [41]:
lattice_vectors = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
])
supa = np.identity(crystal.dim, dtype=int) * 4
pg = "Oh"

In [42]:
kpoints = Kpoints(vec=lattice_vectors, supa=supa, pg=pg)
kpoints.find_irreducible_lattice_points()

The list of irreducible k-points:

In [43]:
kpoints.irreducible_kpoints

array([[Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
       [Fraction(1, 4), Fraction(0, 1), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)],
       [Fraction(1, 4), Fraction(1, 4), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(1, 4), Fraction(0, 1)],
       [Fraction(3, 4), Fraction(1, 4), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(1, 2), Fraction(0, 1)],
       [Fraction(3, 4), Fraction(1, 2), Fraction(1, 4)]], dtype=object)

The list of point group operations that transform a k-point in FTG to it's irreducible counterpart:

In [44]:
kpoints.irreducible_kpoints_trans

array(['E', 'E', 'E', 'I', 'Ic2b', 'E', 'E', 'E', 'Ic2b', 'Ic2b', 'E',
       'Ic2z', 'Ic2z', 'I', 'I', 'I', 'Ic2d', 'Ic2f', 'Ic2f', 'Ic2f',
       'Ic2d', 'Ic2x', 'Ic3ga', 'Ic3ga', 'Ici4y', 'Ic2e', 'Ic2x', 'E',
       'Ic3be', 'Ici3al', 'Ic2b', 'Ic4z', 'Ic2d', 'Ic4x', 'Ic2e',
       'Ici3de', 'Ic2d', 'Ic2y', 'Ici3be', 'Ic2f', 'Ic2c', 'Ic3al',
       'Ic2e', 'Ic4y', 'Ic3be', 'Ic2e', 'Ic2e', 'Ic2x', 'Ic2y', 'Ic4x',
       'Ici3ga', 'Ic2e', 'Ic2d', 'Ic2y', 'Ic2a', 'Ici4x', 'Ic3de', 'I',
       'Ici4z', 'Ici3be', 'Ic2c', 'Ic2c', 'Ic2c', 'Ic2e'], dtype='<U7')

The indices of irreducible k-point each k-point can be rotated into:

In [45]:
kpoints.irreducible_kpoints_map

array([0, 1, 2, 1, 1, 3, 4, 5, 2, 4, 6, 4, 1, 5, 4, 3, 1, 3, 4, 5, 3, 1,
       5, 4, 4, 5, 4, 7, 5, 4, 7, 4, 2, 4, 6, 4, 4, 5, 4, 7, 6, 4, 2, 4,
       4, 7, 4, 5, 1, 5, 4, 3, 5, 4, 7, 4, 4, 7, 4, 5, 3, 4, 5, 1])

Next, we can use these information to find irreducible Q-points.

In [46]:
qpointsn = QpointsN(vec=lattice_vectors, supa=supa, order=3, pg=pg)
qpointsn.find_irreducible_Qpoints()

In [47]:
qpointsn.irreducible_Qpoints

array([[[Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]],

       [[Fraction(3, 4), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(1, 4), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]],

       [[Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]],

       [[Fraction(3, 4), Fraction(3, 4), Fraction(0, 1)],
        [Fraction(1, 4), Fraction(1, 4), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]],

       [[Fraction(1, 2), Fraction(3, 4), Fraction(0, 1)],
        [Fraction(1, 2), Fraction(1, 4), Fraction(0, 1)],
        [Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)]],

       [[Fraction(1, 4), Fraction(3, 4), Fraction(0, 1)],
        [Fraction(3, 4), Fraction(1, 4), Fraction(0, 1)],
    